In [ ]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

In [ ]:
# import opendatasets as od

In [ ]:
#od.download("https://www.kaggle.com/datasets/kaustubhb999/tomatoleaf")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alvinfernando
Your Kaggle Key: ········


100%|███████████████████████████████████████████████████████████████████████████████| 179M/179M [01:40<00:00, 1.86MB/s]


In [ ]:
#FOR LEAF CLASSIFICATION

import zipfile
from google.colab import drive
drive.mount('/content/gdrive')

local_zip = '/content/gdrive/MyDrive/Capstone/LeafDataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale=1.0/255.,
                                     rotation_range=180,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
    
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=64, 
                                                      class_mode='categorical',
                                                      target_size=(150, 150))
    
  validation_datagen = ImageDataGenerator( rescale = 1.0/255.)
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=64, 
                                                                class_mode='categorical',
                                                                target_size=(150, 150))
  return train_generator, validation_generator

In [ ]:
# train_dir = 'tomatoleaf/tomato/train'
# val_dir = 'tomatoleaf/tomato/val'
# train_generator, validation_generator = train_val_generators(train_dir, val_dir)

FileNotFoundError: ignored

In [ ]:
#FOR LEAF CLASSIFICATION

train_dir = '/content/LeafDataset/train'
val_dir = '/content/LeafDataset/test'
train_generator, validation_generator = train_val_generators(train_dir, val_dir)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:
import keras
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.applications.efficientnet_v2 import EfficientNetV2M

base_model = EfficientNetV2M(include_top = False,
                             weights='imagenet',
                             input_shape=(150,150,3))
base_model.summary()

Model: "efficientnetv2-m"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 150, 150, 3)  0           ['input_3[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 75, 75, 24)   648         ['rescaling_2[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 75, 75, 24)   96          ['stem_conv[0][0]'

In [ ]:
# def output_of_last_layer(pre_trained_model):
#   last_desired_layer = pre_trained_model.get_layer('block7e_dwconv2')
#   print('last layer output shape: ', last_desired_layer.output_shape)
#   last_output = last_desired_layer.output
#   print('last layer output: ', last_output)

In [ ]:
# last_output = output_of_last_layer(base_model)

last layer output shape:  (None, 5, 5, 3072)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 5, 5, 3072), dtype=tf.float32, name=None), name='block7e_dwconv2/depthwise:0', description="created by layer 'block7e_dwconv2'")


In [ ]:
# from keras.layers import BatchNormalization
# from keras import Model

# def create_final_model(pre_trained_model):
#   x = last_output
#   x = Flatten()(x)
#   x = Dense(128, activation='relu')(x)
#   x = BatchNormalization()(x)
#   x = Dropout(0.3)(x)
#   x = Dense(3, activation='softmax')(x)

#   model = Model(inputs=pre_trained_model.input, outputs=x)
#   model.compile(optimizer = "adam", 
#               loss = 'categorical_crossentropy',
#               metrics = ['accuracy'])
#   return model

In [ ]:
# print(f"The pretrained model has type: {type(last_output)}")
# print(f"The pretrained model has type: {(last_output)}")
# x = tf.keras.models.Sequential()

# x = base_model(tf.keras.Input(shape=(150, 150, 3)))
# x = base_model.get_layer('block7e_activation')
# x = layers.Flatten(x)
# x = Dense(128, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.3)(x)
# x = Dense(3, activation='softmax')(x)

# model = Model(inputs=pre_trained_model.input, outputs=x)
# model.compile(optimizer = "adam", 
#             loss = 'categorical_crossentropy',
#             metrics = ['accuracy'])
# model.summary()

# print(base_model.output)

AttributeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import EfficientNetV2M

# Load pre-trained EfficientNetV2M model
base_model = EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Get the desired layer from the pre-trained model
desired_layer = base_model.get_layer('block7e_dwconv2')

# Freeze the base model layers
base_model.trainable = False

# Create your additional layers
x = desired_layer.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
outputs = Dense(3, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 150, 150, 3)  0           ['input_4[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 75, 75, 24)   648         ['rescaling_3[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 75, 75, 24)   96          ['stem_conv[0][0]']        

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
acc_ckp_path = '/content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5'
acc_checkpoint = tf.keras.callbacks.ModelCheckpoint(acc_ckp_path, monitor='accuracy', verbose=2,
    save_best_only=True, mode='auto', save_freq='epoch')

val_acc_ckp_path = '/content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5'
val_acc_checkpoint = tf.keras.callbacks.ModelCheckpoint(val_acc_ckp_path, monitor='val_accuracy', verbose=2,
    save_best_only=True, mode='auto', save_freq='epoch')

class stopEpoch(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= 0.95 or (logs.get('accuracy') > 0.92 and logs.get('val_accuracy') > 0.90)):
      print("\nThreshold achieve!\n")
      self.model.stop_training = True

# class SaveBestAccuracyCallback(tf.keras.callbacks.Callback):
#     def __init__(self, save_path):
#         super(SaveBestAccuracyCallback, self).__init__()
#         self.save_path = save_path
#         self.best_accuracy = 0.0

#     def on_epoch_end(self, epoch, logs=None):
#         current_accuracy = logs.get('accuracy')
#         if current_accuracy > self.best_accuracy:
#             self.best_accuracy = current_accuracy
#             self.model.save(self.save_path)  # Save the model

# # Create an instance of the callback
# save_callback = SaveBestAccuracyCallback('/content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5')


In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 100,
                    verbose = 2,
                    callbacks=[acc_checkpoint, val_acc_checkpoint, stopEpoch()])

Epoch 1/100

Epoch 1: accuracy improved from -inf to 0.46333, saving model to /content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5

Epoch 1: val_accuracy improved from -inf to 0.37333, saving model to /content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5
19/19 - 140s - loss: 1.7334 - accuracy: 0.4633 - val_loss: 2.2963 - val_accuracy: 0.3733 - 140s/epoch - 7s/step
Epoch 2/100

Epoch 2: accuracy improved from 0.46333 to 0.57000, saving model to /content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5

Epoch 2: val_accuracy did not improve from 0.37333
19/19 - 64s - loss: 0.8978 - accuracy: 0.5700 - val_loss: 5.4670 - val_accuracy: 0.3333 - 64s/epoch - 3s/step
Epoch 3/100

Epoch 3: accuracy improved from 0.57000 to 0.59083, saving model to /content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5

Epoch 3: val_accuracy did not improve from 0.37333
19/19 - 60s - loss: 0.8479 - accuracy: 0.5908 - val_loss: 4.2253 - val_accuracy: 0.3333 - 60s/epoch - 3s/step
Epoch 4/100

Epoch 4: accuracy did not imp

In [ ]:
save_model_path = "/content/leaf_model.h5"
model.save(save_model_path)

In [ ]:
# model_test = tf.keras.models.load_model('/content/gdrive/MyDrive/Capstone/leaf_model_acc72.h5')

from PIL import Image
import numpy as np
import cv2

# image = cv2.imread("/content/tomat.jpg", cv2.IMREAD_COLOR)
# image = image/255.0

# image = np.array(Image.open("/content/cabe.jpg").resize((150, 150)).convert("RGB"))
# image = np.expand_dims(image, axis=0)
# print(image.shape)
# predict = model_test.predict(image)
# print(predict)

# threshold = 0.5
# if(predict[0][0] > threshold):
#   print("This is a Chili")
# elif (predict[0][1] > threshold):
#   print("This is a Rice")
# elif (predict[0][2] > threshold):
#   print("This is a Tomato")
# else:
#   print("This is not included in the dataset")

(1, 150, 150, 3)
1/1 [==============================] - 0s 93ms/step
[[9.221980e-01 2.260052e-04 7.757604e-02]]
This is a Chili
